In [1]:
import os
import ion_gen as ig
import structure as st
import re
import numpy as np

In [2]:
cwd = os.getcwd()
xyzDir = f"{cwd}/../XYZ/AMOEBA"
os.chdir(xyzDir)

In [3]:
def insertH2O(coord, atO, atH):
    n = len(coord)
    if n%9 != 0:
        raise LengthWrong("Length is not divisible by 9!")
    at_template = [atO,atH,atH]
    at = []
    for i in range(int(n/3)):
        at += at_template
    if len(at) != n:
        raise LengthWrong("Length of AT is not the same as N!")
    result = []
    index = 0
    for i, num in enumerate(coord):
        result.append(num)
        if (i + 1) % 3 == 0 and index < len(at):
            result.append(at[index])
            index += 1
    return result

In [4]:
# Generate-01

for key,item in ig.catAnEquil.items():
    split = key.split("-")
    ion1 = split[0]
    ion2 = split[1]
    at1 = ig.atomTypeAMOEBA[ion1]
    at2 = ig.atomTypeAMOEBA[ion2]
    index = ig.index[key]
    distances = ig.allTwoBody[key]
    for dist in distances:
        fileName = f"{index}_{key}_{dist:3.2f}.xyz"
        f = open(fileName,"w")
        f.write(ig.ionIonXyz.format(ion1,ion2,at1,at2,item*dist))
        f.close()

In [5]:
# Generate-02

for key,item in ig.ionWaterEquil.items():
    split = key.split("-")
    ion1 = split[0]
    at1 = ig.atomTypeAMOEBA[ion1]
    atO = ig.atomTypeAMOEBA["O"]
    atH = ig.atomTypeAMOEBA["H"]
    index = ig.index[key]
    distances = ig.allTwoBody[key]
    struct = st.ionWaterStruct[key]
    struct = insertH2O(struct,atO,atH)
    for dist in distances:
        fileName = f"{index}_{key}_{dist:3.2f}.xyz"
        input = list((*struct,ion1,item*dist,at1))
        f = open(fileName,"w")
        f.write(ig.ionWaterXyz2.format(*input))
        f.close()

In [6]:
# Generate-03

for key,item in ig.ion2WaterEquil.items():
    split = key.split("-")
    ion1 = split[0]
    at1 = ig.atomTypeAMOEBA[ion1]
    atO = ig.atomTypeAMOEBA["O"]
    atH = ig.atomTypeAMOEBA["H"]
    index = ig.index[key]
    distances = ig.allTwoBody[key]
    for dist in distances:
        struct = list(st.ion2WaterStruct[key])
        d = item*dist
        if ion1 in ["Li", "Na", "K"]:
            delta = d - struct[0]
            d = 0
            struct[0] += delta
            struct[3] += delta
            struct[6] += delta
            struct[9] -= delta
            struct[12] -= delta
            struct[15] -= delta
        struct = insertH2O(struct,atO,atH)
        fileName = f"{index}_{key}_{dist:3.2f}.xyz"
        input = list((ion1,d,at1,*struct))
        f = open(fileName,"w")
        f.write(ig.ion2WaterXyz2.format(*input))
        f.close()

In [7]:
# Generate-04

for key in ig.ionManyWater:
    split = key.split("-")
    ion1 = split[0]
    at1 = ig.atomTypeAMOEBA[ion1]
    atO = ig.atomTypeAMOEBA["O"]
    atH = ig.atomTypeAMOEBA["H"]
    index = ig.index[key]
    nw = int(re.search(r'\d+', key).group())
    struct = st.ionManyWaterStruct[key]
    struct = insertH2O(struct,atO,atH)
    fileName = f"{index}_{key}.xyz"
    if nw == 3:
        xyz = ig.ion3WaterXyz2.format(ion1,0,at1,*struct)
    elif nw == 4:
        xyz = ig.ion4WaterXyz2.format(ion1,0,at1,*struct)
    elif nw == 5:
        xyz = ig.ion5WaterXyz2.format(ion1,0,at1,*struct)
    elif nw == 6:
        xyz = ig.ion6WaterXyz2.format(ion1,0,at1,*struct)
    f = open(fileName,"w")
    f.write(xyz)
    f.close()

In [8]:
# Generate-05

def rotate(vector, ax, deg):
    cos = np.cos(deg * np.pi/180)
    sin = np.sin(deg * np.pi/180)
    
    if ax == "x":
        mat = [
            [ 1.0, 0.0, 0.0],
            [ 0.0, cos,-sin],
            [ 0.0, sin, cos],
        ]
    elif ax == "y":
        mat = [
            [ cos, 0.0, sin],
            [ 0.0, 1.0, 0.0],
            [-sin, 0.0, cos],
        ]
    elif ax == "z":
        mat = [
            [ cos,-sin, 0.0],
            [ sin, cos, 0.0],
            [ 0.0, 0.0, 1.0],
        ]
    else:
        ux = ax[0]
        uy = ax[1]
        uz = ax[2]
        r = np.sqrt(ux*ux + uy*uy + uz*uz)
        ux = ux/r
        uy = uy/r
        uz = uz/r
        mat = [
            [ cos + ux*ux*(1-cos)   , ux*uy*(1-cos) - uz*sin, ux*uz*(1-cos) + uy*sin],
            [ uy*ux*(1-cos) + uz*sin, cos + uy*uy*(1-cos)   , uy*uz*(1-cos) - ux*sin],
            [ uz*ux*(1-cos) - uy*sin, uz*uy*(1-cos) + ux*sin, cos + uz*uz*(1-cos)   ],
        ]

    for i in range(0, len(vector), 3):
        v0 = vector[i+0]
        v1 = vector[i+1]
        v2 = vector[i+2]
        vector[i+0] = mat[0][0] * v0 + mat[0][1] * v1 + mat[0][2] * v2
        vector[i+1] = mat[1][0] * v0 + mat[1][1] * v1 + mat[1][2] * v2
        vector[i+2] = mat[2][0] * v0 + mat[2][1] * v1 + mat[2][2] * v2
    return

for key in ig.ionDegWater:
    split = key.split("-")
    ion1 = split[0]
    at1 = ig.atomTypeAMOEBA[ion1]
    atO = ig.atomTypeAMOEBA["O"]
    atH = ig.atomTypeAMOEBA["H"]
    index = ig.index[key]
    angles = ig.allTwoBody[key]
    struct_key = key.split("_Deg")[0]
    ang1 = float(key.split("Deg")[1])
    dist = ig.ionWaterEquil[struct_key]
    structure = list(st.ionWaterStruct[struct_key])
    rotate(structure, "y", ang1)
    a0 = structure[3]
    a1 = structure[4]
    a2 = structure[5]
    b0 = structure[6]
    b1 = structure[7]
    b2 = structure[8]
    c0 = a1*b2 - a2*b1
    c1 = a2*b0 - a0*b2
    c2 = a0*b1 - a1*b0
    bis = [c0,c1,c2]
    for ang2 in angles:
        struct = structure.copy()
        rotate(struct, bis, ang2)
        struct = insertH2O(struct,atO,atH)
        sign = "+"
        if ang2 < 0:
            sign = "-"
        fileName = f"{index}_{key}_{sign}{str(abs(ang2)).zfill(2)}.xyz"
        input = list((*struct,ion1,dist,at1))
        f = open(fileName,"w")
        f.write(ig.ionWaterXyz2.format(*input))
        f.close()

In [9]:
# Generate-06

def splitDimer(dimer):
    if dimer == "LiF":
        ion1 = "Li"
        ion2 = "F"
    elif dimer == "LiCl":
        ion1 = "Li"
        ion2 = "Cl"
    elif dimer == "LiBr":
        ion1 = "Li"
        ion2 = "Br"
    elif dimer == "NaF":
        ion1 = "Na"
        ion2 = "F"
    elif dimer == "NaCl":
        ion1 = "Na"
        ion2 = "Cl"
    elif dimer == "NaBr":
        ion1 = "Na"
        ion2 = "Br"
    elif dimer == "KF":
        ion1 = "K"
        ion2 = "F"
    elif dimer == "KCl":
        ion1 = "K"
        ion2 = "Cl"
    elif dimer == "KBr":
        ion1 = "K"
        ion2 = "Br"
    return ion1,ion2

for key,item in ig.ion2Ion.items():
    split = key.split("-")
    dimer = split[0]
    ion1,ion2 = splitDimer(dimer)
    ion3 = split[1]
    at1 = ig.atomTypeAMOEBA[ion1]
    at2 = ig.atomTypeAMOEBA[ion2]
    at3 = ig.atomTypeAMOEBA[ion3]
    index = ig.index[key]
    distances = ig.allTwoBody[key]
    if ion1 in ["Li","Na","K"] and ion3 in ["Li","Na","K"]:
        ion = ion1
        at = at1
        ion1 = ion2
        at1 = at2
        ion2 = ion
        at2 = at
    for dist in distances:
        fileName = f"{index}_{key}_{dist:3.2f}.xyz"
        f = open(fileName,"w")
        f.write(ig.ion2IonXyz.format(ion1,at1,ion2,item,at2,ion3,item*dist,at3))
        f.close()

In [10]:
# Generate-07

for key in ig.ionIon:
    split = key.split("-")
    ion1 = split[0]
    ion2 = split[1]
    at1 = ig.atomTypeAMOEBA[ion1]
    at2 = ig.atomTypeAMOEBA[ion2]
    index = ig.index[key]
    distances = ig.allTwoBody[key]
    vdw1 = ig.vdwRadii[ion1]
    vdw2 = ig.vdwRadii[ion2]
    equil = vdw1 + vdw2
    for dist in distances:
        fileName = f"{index}_{key}_{dist:3.2f}.xyz"
        f = open(fileName,"w")
        f.write(ig.ionIonXyz.format(ion1,ion2,at1,at2,equil*dist))
        f.close()

In [11]:
# Generate-08

for key in ig.ionWaterNeq:
    split = key.split("-")
    ion1 = split[0]
    at1 = ig.atomTypeAMOEBA[ion1]
    atO = ig.atomTypeAMOEBA["O"]
    atH = ig.atomTypeAMOEBA["H"]
    index = ig.index[key]
    distances = ig.allTwoBody[key]
    vdw1 = ig.vdwRadii[ion1]
    equil = vdw1 + 1.4
    if ion1 in ["Li","Na","K"]:
        itype = "cation"
    elif ion1 in ["F","Cl","Br"]:
        itype = "anion"
    struct = st.ionWaterNeqStruct[itype]
    struct = insertH2O(struct,atO,atH)
    for dist in distances:
        fileName = f"{index}_{key}_{dist:3.2f}.xyz"
        f = open(fileName,"w")
        f.write(ig.ionWaterXyz2.format(*struct,ion1,equil*dist,at1))
        f.close()

In [12]:
# Generate-09

for key in ig.ion2WaterNeq:
    split = key.split("-")
    ion1 = split[0][1:]
    at1 = ig.atomTypeAMOEBA[ion1]
    atO = ig.atomTypeAMOEBA["O"]
    atH = ig.atomTypeAMOEBA["H"]
    index = ig.index[key]
    distances = ig.allTwoBody[key]
    vdw1 = ig.vdwRadii[ion1]
    equil = vdw1 + 1.4
    if ion1 in ["Li","Na","K"]:
        itype = "cation"
    elif ion1 in ["F","Cl","Br"]:
        itype = "anion"
    struct = st.ion2WaterNeqStruct[itype]
    struct = insertH2O(struct,atO,atH)
    for dist in distances:
        d = np.sqrt((equil*dist)**2/2)
        fileName = f"{index}_{key}_{dist:3.2f}.xyz"
        f = open(fileName,"w")
        f.write(ig.ion2WaterNeqXyz2.format(*struct,ion1,d,d,at1,ion1,d,-d,at1))
        f.close()

In [13]:
# Generate-10

ionTemp = "{:>6d}  {:3}{:12.6f}{:12.6f}{:12.6f}{:>6d}\n"
OTemp = "{:>6d}  {:3}{:12.6f}{:12.6f}{:12.6f}{:>6d}{:>6d}{:>6d}\n"
HTemp = "{:>6d}  {:3}{:12.6f}{:12.6f}{:12.6f}{:>6d}{:>6d}\n"

def replace_atom_type(old_lines, old_atom_type, new_atom_type):
    modified_lines = []
    for line in old_lines:
        parts = line.split()
        parts[0] = int(parts[0])
        if len(parts) >= 6 and parts[1] == old_atom_type:
            parts[2] = float(parts[2])
            parts[3] = float(parts[3])
            parts[4] = float(parts[4])
            parts[5] = new_atom_type
            if old_atom_type == "O":
                parts[6] = int(parts[6])
                parts[7] = int(parts[7])
                line = OTemp.format(*parts)
            elif old_atom_type == "H":
                parts[6] = int(parts[6])
                line = HTemp.format(*parts)
            else:
                line = ionTemp.format(*parts)
        modified_lines.append(line)
    return modified_lines

for key,value in ig.ionClsWater.items():
    split = key.split("-")
    ion = split[0]
    if ion in ["Li","Na","K"]:
        ionname = ion + "+"
    elif ion1 in ["F","Cl","Br"]:
        ionname = ion + "-"
    at = ig.atomTypeAMOEBA[ion]
    atO = ig.atomTypeAMOEBA["O"]
    atH = ig.atomTypeAMOEBA["H"]
    index = ig.index[key]
    frames = ig.allTwoBody[key]
    for frame in frames:
        fn = f"{index}_{key}_{str(frame).zfill(3)}.xyz"
        files = f"{cwd}/../SIM/{value}_{ion}/CLUSTER/{fn}"
        f = open(files, "r")
        lines = f.readlines()
        f.close()
        lines = replace_atom_type(lines, ionname, at)
        lines = replace_atom_type(lines, "O", atO)
        lines = replace_atom_type(lines, "H", atH)
        f = open(fn, "w")
        for line in lines:
            f.write(line)
        f.close()

In [14]:
# Generate-03 Subsets

for key,item in ig.ion2WaterEquil.items():
    split = key.split("-")
    ion1 = split[0]
    at1 = ig.atomTypeAMOEBA[ion1]
    atO = ig.atomTypeAMOEBA["O"]
    atH = ig.atomTypeAMOEBA["H"]
    index = ig.index[key]
    distances = ig.allTwoBody[key]
    for dist in distances:
        struct = list(st.ion2WaterStruct[key])
        d = item*dist
        if ion1 in ["Li", "Na", "K"]:
            delta = d - struct[0]
            d = 0
            struct[0] += delta
            struct[3] += delta
            struct[6] += delta
            struct[9] -= delta
            struct[12] -= delta
            struct[15] -= delta
        struct = insertH2O(struct,atO,atH)
        # 01
        fileName = f"{index}_{key}_{dist:3.2f}_01.xyz"
        input = list((ion1,d,0,0,at1,*struct[:12]))
        f = open(fileName,"w")
        f.write(ig.GenIonWater.format(*input))
        f.close()
        # 02
        fileName = f"{index}_{key}_{dist:3.2f}_02.xyz"
        input = list((ion1,d,0,0,at1,*struct[12:]))
        f = open(fileName,"w")
        f.write(ig.GenIonWater.format(*input))
        f.close()
        # 12
        fileName = f"{index}_{key}_{dist:3.2f}_12.xyz"
        input = struct
        f = open(fileName,"w")
        f.write(ig.Gen2Water.format(*input))
        f.close()
        # 1
        fileName = f"{index}_{key}_{dist:3.2f}_1.xyz"
        input = struct[:12]
        f = open(fileName,"w")
        f.write(ig.GenWater.format(*input))
        f.close()
        # 2
        fileName = f"{index}_{key}_{dist:3.2f}_2.xyz"
        input = struct[12:]
        f = open(fileName,"w")
        f.write(ig.GenWater.format(*input))
        f.close()

In [15]:
# Generate-04 Subsets

for key in ig.ionManyWater:
    split = key.split("-")
    ion1 = split[0]
    at1 = ig.atomTypeAMOEBA[ion1]
    atO = ig.atomTypeAMOEBA["O"]
    atH = ig.atomTypeAMOEBA["H"]
    index = ig.index[key]
    nw = int(re.search(r'\d+', key).group())
    struct = st.ionManyWaterStruct[key]
    struct = insertH2O(struct,atO,atH)
    fileName = f"{index}_{key}_B.xyz"
    if nw == 3:
        xyz = ig.Gen3Water.format(*struct)
    elif nw == 4:
        xyz = ig.Gen4Water.format(*struct)
    elif nw == 5:
        xyz = ig.Gen5Water.format(*struct)
    elif nw == 6:
        xyz = ig.Gen6Water.format(*struct)
    f = open(fileName,"w")
    f.write(xyz)
    f.close()

In [16]:
# Generate-06 Subsets

def splitDimer(dimer):
    if dimer == "LiF":
        ion1 = "Li"
        ion2 = "F"
    elif dimer == "LiCl":
        ion1 = "Li"
        ion2 = "Cl"
    elif dimer == "LiBr":
        ion1 = "Li"
        ion2 = "Br"
    elif dimer == "NaF":
        ion1 = "Na"
        ion2 = "F"
    elif dimer == "NaCl":
        ion1 = "Na"
        ion2 = "Cl"
    elif dimer == "NaBr":
        ion1 = "Na"
        ion2 = "Br"
    elif dimer == "KF":
        ion1 = "K"
        ion2 = "F"
    elif dimer == "KCl":
        ion1 = "K"
        ion2 = "Cl"
    elif dimer == "KBr":
        ion1 = "K"
        ion2 = "Br"
    return ion1,ion2

for key,item in ig.ion2Ion.items():
    split = key.split("-")
    dimer = split[0]
    ion1,ion2 = splitDimer(dimer)
    ion3 = split[1]
    at1 = ig.atomTypeAMOEBA[ion1]
    at2 = ig.atomTypeAMOEBA[ion2]
    at3 = ig.atomTypeAMOEBA[ion3]
    index = ig.index[key]
    distances = ig.allTwoBody[key]
    if ion1 in ["Li","Na","K"] and ion3 in ["Li","Na","K"]:
        ion = ion1
        at = at1
        ion1 = ion2
        at1 = at2
        ion2 = ion
        at2 = at
    for dist in distances:
        # 01
        fileName = f"{index}_{key}_{dist:3.2f}_01.xyz"
        f = open(fileName,"w")
        f.write(ig.GenIonIon.format(ion1,0,0,0,at1,ion2,0,item,0,at2))
        f.close()
        # 02
        fileName = f"{index}_{key}_{dist:3.2f}_02.xyz"
        f = open(fileName,"w")
        f.write(ig.GenIonIon.format(ion1,0,0,0,at1,ion3,item*dist,0,0,at3))
        f.close()
        # 12
        fileName = f"{index}_{key}_{dist:3.2f}_12.xyz"
        f = open(fileName,"w")
        f.write(ig.GenIonIon.format(ion2,0,item,0,at2,ion3,item*dist,0,0,at3))
        f.close()

In [ ]:
# Generate-09 Subsets

for key in ig.ion2WaterNeq:
    split = key.split("-")
    ion1 = split[0][1:]
    at1 = ig.atomTypeAMOEBA[ion1]
    atO = ig.atomTypeAMOEBA["O"]
    atH = ig.atomTypeAMOEBA["H"]
    index = ig.index[key]
    distances = ig.allTwoBody[key]
    vdw1 = ig.vdwRadii[ion1]
    equil = vdw1 + 1.4
    if ion1 in ["Li","Na","K"]:
        itype = "cation"
    elif ion1 in ["F","Cl","Br"]:
        itype = "anion"
    struct = st.ion2WaterNeqStruct[itype]
    struct = insertH2O(struct,atO,atH)
    for dist in distances:
        d = np.sqrt((equil*dist)**2/2)
        #01
        fileName = f"{index}_{key}_{dist:3.2f}_01.xyz"
        f = open(fileName,"w")
        f.write(ig.ionWaterXyz3.format(*struct,ion1,d,d,0,at1))
        f.close()
        #02
        fileName = f"{index}_{key}_{dist:3.2f}_02.xyz"
        f = open(fileName,"w")
        f.write(ig.ionWaterXyz3.format(*struct,ion1,d,d,0,at1))
        f.close()